In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
X.shape, y.shape, len(set(y))

In [ ]:
X = pd.DataFrame(X, columns=['feature_1', 'feature_2', 'feature_3', 'feature_4'])
y = pd.DataFrame(y, columns=['target'])
X.head()

# Подготовка и визуализация датасета

In [ ]:
X.duplicated().sum()

In [ ]:
ind = X[X.duplicated()].index
X.drop_duplicates(inplace=True)
y.drop(ind, inplace=True)

In [ ]:
X.isna().sum()

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(16, 5))
for i in range(1, 5):
    axs[i - 1].hist(X[f'feature_{i}'], bins=16)
    axs[i - 1].set_title(f'feature - {i}')

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(22, 6))
for i in range(4):
    axs[i].set_xlabel(f'feature_{i + 1}')
    axs[i].set_ylabel('target')
    axs[i].scatter(x=X[f'feature_{i + 1}'], y=y)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(22, 6))
for i in range(4):
    axs[i].set_xlabel(f'feature_{i + 1}')
    axs[i].boxplot(x=X[f'feature_{i + 1}'])

In [ ]:
X.describe()

# Тест на нормальное распределение и обучение модели

In [ ]:
from scipy.stats import shapiro


for i in range(4):
    pvalue = shapiro(X[f'feature_{i + 1}']).pvalue
    print(f'feature_{i + 1}:', pvalue, 'norm distribution' if pvalue >= 0.05 else 'other distribution')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Ковариация и визуализация

In [ ]:
cov_matrix_class1 = np.cov(X_train.loc[(y_train == 0).index, :].T)
cov_matrix_class2 = np.cov(X_train.loc[(y_train == 1).index, :].T)
cov_matrix_class3 = np.cov(X_train.loc[(y_train == 2).index, :].T)

fig, axs = plt.subplots(4, 2, figsize=(10, 12))

im1 = axs[0, 0].imshow(cov_matrix_class1, interpolation='nearest')
axs[0, 0].set_title('Class 1 - Scalar')
plt.colorbar(im1, ax=axs[0, 0])

im2 = axs[0, 1].imshow(np.diag(np.diag(cov_matrix_class1)), interpolation='nearest')
axs[0, 1].set_title('Class 1 - Diagonal')
plt.colorbar(im2, ax=axs[0, 1])

im3 = axs[1, 0].imshow(cov_matrix_class2, interpolation='nearest')
axs[1, 0].set_title('Class 2 - Scalar')
plt.colorbar(im3, ax=axs[1, 0])

im4 = axs[1, 1].imshow(np.diag(np.diag(cov_matrix_class2)), interpolation='nearest')
axs[1, 1].set_title('Class 2 - Diagonal')
plt.colorbar(im4, ax=axs[1, 1])

im5 = axs[2, 0].imshow(cov_matrix_class3, interpolation='nearest')
axs[2, 0].set_title('Class 3 - Scalar')
plt.colorbar(im5, ax=axs[2, 0])

im6 = axs[2, 1].imshow(np.diag(np.diag(cov_matrix_class3)), interpolation='nearest')
axs[2, 1].set_title('Class 3 - Diagonal')
plt.colorbar(im6, ax=axs[2, 1])

im7 = axs[3, 0].imshow(np.mean([cov_matrix_class1, cov_matrix_class2, cov_matrix_class3], axis=0), interpolation='nearest')
axs[3, 0].set_title('Classes matrix equal - Scalar')
plt.colorbar(im7, ax=axs[3, 0])

im8 = axs[3, 1].imshow(np.diag(np.diag(X_train.iloc[:, :])), interpolation='nearest')
axs[3, 1].set_title('Classes matrix equal  - Diagonal')
plt.colorbar(im8, ax=axs[3, 1])

fig.tight_layout()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd



fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_train.iloc[:, 0].values, X_train.iloc[:, 1].values, X_train.iloc[:, 2].values, c=y_train.values, vmin=0.0, vmax=2)

ax.set_xlabel('feature_1')
ax.set_ylabel('feature_2')
ax.set_zlabel('feature_3')

plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_train.iloc[:, 1].values, X_train.iloc[:, 2].values, X_train.iloc[:, 3].values, c=y_train.values, vmin=0.0, vmax=2)

ax.set_xlabel('feature_2')
ax.set_ylabel('feature_3')
ax.set_zlabel('feature_4')

plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_train.iloc[:, 0].values, X_train.iloc[:, 1].values, X_train.iloc[:, 3].values, c=y_train.values, vmin=0.0, vmax=2)

ax.set_xlabel('feature_1')
ax.set_ylabel('feature_2')
ax.set_zlabel('feature_4')

plt.show()

# Метрики

In [ ]:
from sklearn.metrics import accuracy_score


preds = gnb.predict(X_test)
acc = accuracy_score(y_test, preds)
acc

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape

In [ ]:
from sklearn.metrics import RocCurveDisplay


preds = gnb.predict_proba(X_test)
RocCurveDisplay.from_predictions(
    y_onehot_test.ravel(),
    preds.ravel(),
    name="micro-average OvR",
    color="darkorange",
)
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

micro_roc_auc_ovr = roc_auc_score(
    y_test,
    preds,
    multi_class="ovr",
    average="micro",
)

print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")

In [ ]:
from sklearn.model_selection import cross_validate


clf = GaussianNB()
cv_results = cross_validate(clf, X, y, cv=3, scoring='f1_macro')
cv_results['test_score'].mean()

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(y_test, preds, labels=[0, 1, 2]))

# Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression


model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
preds = model.predict_proba(X_test)

In [ ]:
micro_roc_auc_ovr = roc_auc_score(
    y_test,
    preds,
    multi_class="ovr",
    average="micro",
)

print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")

# Отчет


В ходе выполнения работы был использован датасет ирисов из стандартной библиотеки sklearn для многоклассовой классификации.


Первым шагом были визуализированы графики признаков для понимания распределения данных, количестве выбросов. Также были удалены дубликаты из датасета и выполнена проверка на nan значения.


Далее по тесту признаков на нормальное распределение только 2-й признак соответствует нормальному распределению(p_value > 0.05).


Следующим шагом можно увидеть обучение байесовского классификатора и визуализация ковариационных матриц, исходя из различных предположений.


Далее было оценены значения метрик для байесовского классификатора(acc = 0.93, micro_averaged_f1 = 0.99) и приведены соответсвующие графики, в том числе расширенный classification report. Также был рассмотрен случай кросс-валидации данных, который показывает примерно такой же результат, что и при hold-out разбиении (acc=0.94).


И в заключении приведена модель логистической регресии, которая показала более высокие результаты метрики, в отличии от байесовского классификатора